# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

In [1]:
# Installing Theano
! pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

# Installing Tensorflow
! pip install --upgrade tensorflow

# Installing Keras
! pip install --upgrade keras

# Install other packages
! pip install --upgrade pip pandas numpy matplotlib sklearn

/bin/sh: 1: pip: not found
/bin/sh: 1: pip: not found
/bin/sh: 1: pip: not found
/bin/sh: 1: pip: not found


# Parte 1 - Preprocessamento dos Dados

### Importar as libs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]
print("X >>\n",X)

X >>
 [[0.0000000e+00 0.0000000e+00 6.1900000e+02 ... 1.0000000e+00
  1.0000000e+00 1.0134888e+05]
 [0.0000000e+00 1.0000000e+00 6.0800000e+02 ... 0.0000000e+00
  1.0000000e+00 1.1254258e+05]
 [0.0000000e+00 0.0000000e+00 5.0200000e+02 ... 1.0000000e+00
  0.0000000e+00 1.1393157e+05]
 ...
 [0.0000000e+00 0.0000000e+00 7.0900000e+02 ... 0.0000000e+00
  1.0000000e+00 4.2085580e+04]
 [1.0000000e+00 0.0000000e+00 7.7200000e+02 ... 1.0000000e+00
  0.0000000e+00 9.2888520e+04]
 [0.0000000e+00 0.0000000e+00 7.9200000e+02 ... 1.0000000e+00
  0.0000000e+00 3.8190780e+04]]


/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

---
# Parte 2 -Vamos construir uma ANN!

## Importando o Keras


In [7]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [8]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Ajustando a ANN ao conjunto de treinamento

In [9]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 172us/step - loss: 0.4842 - accuracy: 0.7958
Epoch 2/100
8000/8000 [==============================] - 1s 109us/step - loss: 0.4272 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 102us/step - loss: 0.4225 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 106us/step - loss: 0.4189 - accuracy: 0.8127
Epoch 5/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.4166 - accuracy: 0.8263
Epoch 6/100
8000/8000 [==============================] - 1s 103us/step - loss: 0.4148 - accuracy: 0.8284
Epoch 7/100
8000/8000 [==============================] - 1s 108us/step - loss: 0.4136 - accuracy: 0.8300
Epoch 8/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4121 - accuracy: 0.8305
Epoch 9/100
8000/8000 [==============================] - 1s 119us/step - loss: 0.4112 - accuracy: 0.8316
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 1s 133us/step - loss: 0.4007 - accuracy: 0.8357
Epoch 79/100
8000/8000 [==============================] - 1s 121us/step - loss: 0.4008 - accuracy: 0.8360
Epoch 80/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.4004 - accuracy: 0.8351
Epoch 81/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.4007 - accuracy: 0.8359
Epoch 82/100
8000/8000 [==============================] - 1s 129us/step - loss: 0.4008 - accuracy: 0.8363
Epoch 83/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4007 - accuracy: 0.8355
Epoch 84/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4007 - accuracy: 0.8344
Epoch 85/100
8000/8000 [==============================] - 1s 119us/step - loss: 0.4005 - accuracy: 0.8346
Epoch 86/100
8000/8000 [==============================] - 1s 121us/step - loss: 0.4005 - accuracy: 0.8342
Epoch 87/100
8000/8000 [==============================] - 1

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Criando uma Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1510   85]
 [ 196  209]]


# Initiate activation function experimentation

# Initialising the 2nd ANN

In [14]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 2s 242us/step - loss: 3.2881 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 2s 191us/step - loss: 3.2881 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 194us/step - loss: 3.2881 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 254us/step - loss: 3.2881 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 2s 218us/step - loss: 3.2881 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 187us/step - loss: 3.2881 - acc: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 185us/step - loss: 3.2881 - acc: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 180us/step - loss: 3.2881 - acc: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 183us/step - loss: 3.2881 - acc: 0.7960
Epoch 10/100
8000/8000 [==============================] - 2s 190us/step - loss: 3.2881 - acc: 0.7960

In [10]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [11]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1542   53]
 [ 262  143]]


# Initialising the 3rd ANN

In [13]:
del classifier

classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'tanh'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 135us/step - loss: 0.5129 - accuracy: 0.7959
Epoch 2/100
8000/8000 [==============================] - 1s 105us/step - loss: 0.4544 - accuracy: 0.7961
Epoch 3/100
8000/8000 [==============================] - 1s 121us/step - loss: 0.4512 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4515 - accuracy: 0.7958
Epoch 5/100
8000/8000 [==============================] - 1s 138us/step - loss: 0.4569 - accuracy: 0.7956
Epoch 6/100
8000/8000 [==============================] - 1s 144us/step - loss: 0.4550 - accuracy: 0.7959
Epoch 7/100
8000/8000 [==============================] - 1s 109us/step - loss: 0.4493 - accuracy: 0.7960
Epoch 8/100
8000/8000 [==============================] - 1s 133us/step - loss: 0.4661 - accuracy: 0.7960
Epoch 9/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4580 - accuracy: 0.7960
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 1s 117us/step - loss: 0.4165 - accuracy: 0.8217
Epoch 79/100
8000/8000 [==============================] - 1s 112us/step - loss: 0.4130 - accuracy: 0.8216
Epoch 80/100
8000/8000 [==============================] - 1s 128us/step - loss: 0.4183 - accuracy: 0.8209
Epoch 81/100
8000/8000 [==============================] - 1s 129us/step - loss: 0.4062 - accuracy: 0.8238
Epoch 82/100
8000/8000 [==============================] - 1s 133us/step - loss: 0.4414 - accuracy: 0.8311
Epoch 83/100
8000/8000 [==============================] - 1s 138us/step - loss: 0.4140 - accuracy: 0.8216
Epoch 84/100
8000/8000 [==============================] - 1s 133us/step - loss: 0.4207 - accuracy: 0.8354
Epoch 85/100
8000/8000 [==============================] - 1s 115us/step - loss: 0.4256 - accuracy: 0.8443
Epoch 86/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4172 - accuracy: 0.8438
Epoch 87/100
8000/8000 [==============================] - 1

In [14]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix

In [15]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1557   38]
 [ 261  144]]


---
# Parte 4 - Evaluating, Improving and Tuning the ANN

## Evaluating the ANN

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()
print("accuracies:%s\nmean:%s\nvariance:%s".format(accuracies, mean, variance))

accuracies:%s
mean:%s
variance:%s


# Improving the ANN
# Dropout Regularization to reduce overfitting if needed

In [17]:
from keras.layers import Dropout

def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 10)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

# Tuning the ANN

In [18]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh', input_dim = 11))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'tanh'))
    classifier.add(Dropout(rate = 0.1))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {'batch_size': [2, 5, 8],
              'epochs': [10, 10, 10],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           scoring = 'accuracy',
                           param_grid = parameters,
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)